In [3]:
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 2 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM ubuntu:16.04

MAINTAINER Amazon AI <sage-learner@amazon.com>

RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python \
         python3.5 \
         nginx \
         ca-certificates \
         libgcc-5-dev \
    && rm -rf /var/lib/apt/lists/*


# Symlink /usr/bin/python to the python version we're building for.
RUN rm /usr/bin/python && ln -s /usr/bin/python3.5 /usr/bin/python

# Here we get all python packages.
# There's substantial overlap between scipy and numpy that we eliminate by
# linking them together. Likewise, pip leaves the install caches populated which uses
# a significant amount of space. These optimizations save a fair amount of space in the
# image, which reduces start up time.
RUN wget https://bootstrap.pypa.io/3.3/get-pip.py && python3.5 get-

### Building and registering the container

The following shell code shows how to build the container image using `docker build` and push the container image to ECR using `docker push`. This code is also available as the shell script `container/build-and-push.sh`, which you can run as `build-and-push.sh decision_trees_sample` to build the image `decision_trees_sample`. 

This code looks for an ECR repository in the account you're using and the current default region (if you're using a SageMaker notebook instance, this will be the region where the notebook instance was created). If the repository doesn't exist, the script will create it.

In [4]:
%%sh

# The name of our algorithm
algorithm_name=gamma-migration

cd container

chmod +x gamma-migration/train
chmod +x gamma-migration/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded

Step 1/10 : FROM ubuntu:16.04
 ---> 9361ce633ff1
Step 2/10 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> 2caa558979bb
Step 3/10 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python          python3.5          nginx          ca-certificates          libgcc-5-dev     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> f06b8407c0a6
Step 4/10 : RUN rm /usr/bin/python && ln -s /usr/bin/python3.5 /usr/bin/python
 ---> Using cache
 ---> 2bb0523ec33a
Step 5/10 : RUN wget https://bootstrap.pypa.io/3.3/get-pip.py && python3.5 get-pip.py &&  pip3 install numpy scipy scikit-learn statsmodels pandas flask gevent gunicorn &&  (cd /usr/local/lib/python3.5/dist-packages/scipy/.libs; rm *; ln ../../numpy/.libs/* .) &&  rm -rf /root/.cache
 ---> Using cache
 ---> 4dbaaab999e3
Step 6/10 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 71072a6942b1
Step 7/10 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



## Testing your algorithm on your local machine or on an Amazon SageMaker notebook instance

While you're first packaging an algorithm use with Amazon SageMaker, you probably want to test it yourself to make sure it's working right. In the directory `container/local_test`, there is a framework for doing this. It includes three shell scripts for running and using the container and a directory structure that mimics the one outlined above.

The scripts are:

* `train_local.sh`: Run this with the name of the image and it will run training on the local tree. You'll want to modify the directory `test_dir/input/data/...` to be set up with the correct channels and data for your algorithm. Also, you'll want to modify the file `input/config/hyperparameters.json` to have the hyperparameter settings that you want to test (as strings).
* `serve_local.sh`: Run this with the name of the image once you've trained the model and it should serve the model. It will run and wait for requests. Simply use the keyboard interrupt to stop it.
* `predict.sh`: Run this with the name of a payload file and (optionally) the HTTP content type you want. The content type will default to `text/csv`. For example, you can run `$ ./predict.sh payload.csv text/csv`.

The directories as shipped are set up to test the decision trees sample algorithm presented here.

# Part 2: Using your Algorithm in Amazon SageMaker

Once you have your container packaged, you can use it to train models and use the model for hosting or batch transforms. Let's do that with the algorithm we made above.

## Set up the environment

Here we specify a bucket to use and the role that will be used for working with SageMaker.

In [5]:
# S3 prefix
prefix = 'gamma-migration'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## Create the session

The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [6]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## Upload the data for training

When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. For the purposes of this example, we're using some the classic [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set), which we have included. 

We can use use the tools provided by the SageMaker Python SDK to upload the data to a default bucket. 

In [7]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-452432741922


In [8]:
data_location

's3://sagemaker-us-east-1-452432741922/gamma-migration'

## Create an estimator and fit the model

In order to use SageMaker to fit our algorithm, we'll create an `Estimator` that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:

* The __container name__. This is constructed as in the shell commands above.
* The __role__. As defined above.
* The __instance count__ which is the number of machines to use for training.
* The __instance type__ which is the type of machine to use for training.
* The __output path__ determines where the model artifact will be written.
* The __session__ is the SageMaker session object that we defined above.

Then we use fit() on the estimator to train against the data that we uploaded above.

In [9]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/gamma-migration:latest'.format(account, region)

tree = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_location)

INFO:sagemaker:Creating training-job with name: gamma-migration-2019-04-08-14-12-38-180


2019-04-08 14:12:38 Starting - Starting the training job...
2019-04-08 14:12:39 Starting - Launching requested ML instances......
2019-04-08 14:13:47 Starting - Preparing the instances for training......
2019-04-08 14:15:08 Downloading - Downloading input data
2019-04-08 14:15:08 Training - Training image download completed. Training in progress.
2019-04-08 14:15:08 Uploading - Uploading generated training model.
Starting the training.
['/opt/ml/input/data/training/test.csv', '/opt/ml/input/data/training/train.csv']
/opt/program/train:53: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  train_data = pd.concat(raw_data)
Training complete.

2019-04-08 14:15:14 Completed - Training job completed
Billable seconds: 33


## Hosting your model
You can use a trained model to get real time predictions using HTTP endpoint. Follow these steps to walk you through the process.

### Deploy the model

Deploying the model to SageMaker hosting just requires a `deploy` call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

In [ ]:
from sagemaker.predictor import csv_serializer
predictor = tree.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

### Choose some data and use it for a prediction

In order to do some predictions, we'll extract some of the data we used for training and do predictions against it. This is, of course, bad statistical practice, but a good way to see how the mechanism works.

In [ ]:
test_data = pd.read_csv("data/test.csv")[0:100]

In [ ]:
test_data.to_csv('data/test_sample.csv')

In [ ]:
import boto3

client = boto3.client('sagemaker-runtime')

endpoint_name = "gamma-migration-2019-04-01-18-03-12-104"                                       # Your endpoint name.
content_type = "text/csv"                                        # The MIME type of the input data in the request body.

In [ ]:
%%timeit
response = client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType=content_type,
    Body=open('data/test_sample.csv', 'rb')
    )

In [ ]:
response = client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType=content_type,
    Body=open('data/test_sample.csv', 'rb')
    )

print(response)                         # If model receives and updates the custom_attributes header 
                        

In [ ]:
print(response['Body'].read())

### Optional cleanup
When you're done with the endpoint, you'll want to clean it up.

In [ ]:
sess.delete_endpoint('gamma-migration-2019-04-01-18-03-12-104')